# 머신러닝 파이프라인과 GridSearchCV를 이용한 하이퍼파라미터 최적화

이 노트북은 `scikit-learn`의 **`Pipeline`**과 **`GridSearchCV`**를 함께 사용하여 머신러닝 워크플로우를 효율적으로 구축하고 하이퍼파라미터를 최적화하는 방법을 다룹니다.

### `Pipeline`이란?
`Pipeline`은 여러 전처리 단계와 최종 모델을 하나로 묶어주는 도구입니다. 이를 통해 코드를 간결하게 만들고, 데이터 전처리 과정에서 발생할 수 있는 **데이터 누수(Data Leakage)**를 방지하며, 워크플로우를 체계적으로 관리할 수 있습니다.

### `GridSearchCV`란?
`GridSearchCV`는 사용자가 지정한 하이퍼파라미터 값들의 모든 가능한 조합에 대해 모델을 학습시키고 교차 검증(Cross-Validation)을 수행하여, 가장 좋은 성능을 보이는 조합을 찾아주는 도구입니다.

### 1. 라이브러리 임포트 및 데이터 로드

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_breast_cancer 
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.pipeline import Pipeline 
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score # roc_auc_score 추가

In [ ]:
# 유방암 데이터셋 로드
cancer = load_breast_cancer() 
X = cancer.data 
y = cancer.target 

print(f"데이터 형태: {X.shape}")
print(f"타겟 형태: {y.shape}")
print(f"클래스 분포: {np.bincount(y)}") # 0과 1의 개수 확인

### 2. 데이터 분할

모델 학습과 평가를 위해 데이터를 훈련 세트와 테스트 세트로 분할합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234)

print(f"훈련 데이터 형태: {X_train.shape}")
print(f"테스트 데이터 형태: {X_test.shape}")

### 3. Pipeline 구축

데이터 스케일링(`StandardScaler`)과 로지스틱 회귀 모델(`LogisticRegression`)을 하나의 파이프라인으로 연결합니다.

In [ ]:
pipeline = Pipeline(
    steps=[
        ('scaler', StandardScaler()), # 첫 번째 단계: 스케일러
        ('classifier', LogisticRegression(random_state=42)) # 두 번째 단계: 분류기
    ]
) 

print("구축된 파이프라인:")
print(pipeline)

### 4. GridSearchCV 설정

`GridSearchCV`를 사용하여 파이프라인 내의 하이퍼파라미터를 탐색합니다.

- **`param_grid`**: 파이프라인 내의 각 단계의 하이퍼파라미터는 `단계이름__매개변수` 형식으로 지정합니다. (예: `scaler__` 또는 `classifier__`)
- **`scaler`**: `StandardScaler`와 `MinMaxScaler` 중 어떤 스케일러를 사용할지 탐색합니다.
- **`classifier__C`**: 로지스틱 회귀의 규제 강도 `C`를 탐색합니다.
- **`classifier__solver`**: 로지스틱 회귀의 최적화 알고리즘 `solver`를 탐색합니다.
- **`cv`**: `StratifiedKFold`를 사용하여 클래스 비율을 유지하면서 교차 검증을 수행합니다. 이는 불균형 데이터셋에 특히 중요합니다.
- **`scoring`**: 모델 평가 지표로 `roc_auc`를 사용합니다. `roc_auc`는 이진 분류에서 모델의 전반적인 성능을 잘 나타내는 지표입니다.

In [ ]:
param_grid = {
    'scaler':[StandardScaler(), MinMaxScaler()], # 스케일러 선택
    'classifier__C':[0.01, 0.1, 10, 100], # 로지스틱 회귀의 C 매개변수
    'classifier__solver':['liblinear', 'lbfgs'] # 로지스틱 회귀의 solver 매개변수
}

# StratifiedKFold를 사용하여 교차 검증 폴드 설정
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=pipeline, 
    param_grid=param_grid,
    cv = cv_strategy,
    scoring = 'roc_auc', # ROC AUC 점수를 기준으로 최적화
    n_jobs=-1, # 모든 CPU 코어 사용
    verbose=2  # 학습 과정 상세 출력
)

print("GridSearchCV 설정 완료.")

### 5. 하이퍼파라미터 튜닝 실행

`grid_search.fit()`을 호출하여 훈련 데이터에 대한 최적화 과정을 시작합니다. `GridSearchCV`는 `param_grid`에 정의된 모든 조합에 대해 교차 검증을 수행합니다.

In [ ]:
print("GridSearchCV 학습 시작...")
grid_search.fit(X_train, y_train)
print("GridSearchCV 학습 완료.")

### 6. 결과 분석

`GridSearchCV`가 찾아낸 최적의 하이퍼파라미터 조합과 해당 교차 검증 점수를 확인합니다.

In [ ]:
print("
최적의 파라미터 조합:")
print(grid_search.best_params_)

print("
최고 교차 검증 점수 (ROC AUC):")
print(f"{grid_search.best_score_:.4f}")

# 최적 모델로 테스트 세트 평가
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
test_roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"
테스트 세트 정확도: {test_accuracy:.4f}")
print(f"테스트 세트 ROC AUC: {test_roc_auc:.4f}")

print("
--- 결론 ---")
print("Pipeline과 GridSearchCV를 함께 사용하면 데이터 전처리부터 모델 선택, 하이퍼파라미터 튜닝까지의 복잡한 과정을 체계적이고 효율적으로 수행할 수 있습니다.")
print("특히 StratifiedKFold와 roc_auc와 같은 적절한 교차 검증 전략 및 평가 지표를 사용하면 불균형 데이터셋에서도 모델의 성능을 정확하게 평가하고 최적화할 수 있습니다.")